In [1]:
import os
os.environ['HF_HOME'] = '/data1/malto/cache'

# Using a Quantized LLM and Few-Shot Learning to Generate Synthetic Labels

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from pathlib import Path
from random import random
from datasets import load_dataset

#model_name_or_path = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"
#revision = "gptq-3bit-128g-actorder_True"

model_name_or_path = "TheBloke/SOLAR-10.7B-Instruct-v1.0-GPTQ"
revision = "gptq-8bit-32g-actorder_True"

# tasks MT, DM, PG

TASK_TYPE = "PG"
BATCH_SIZE = 40
BASE_DIR = Path("/data1/malto/shroom")

# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision=revision)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

if tokenizer.pad_token == None: # apparently Mixtral does not have a padding token in tokenizer
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

In [3]:
ds = load_dataset("json", data_files=["/data1/malto/shroom/val.model-agnostic.json"]).shuffle()
ds['train'] = ds['train'].filter(lambda x: x['task'] == TASK_TYPE)
ds['train']

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'label', 'model', 'ref', 'hyp', 'task', 'tgt', 'p(Hallucination)', 'src'],
    num_rows: 125
})

In [4]:
num_examples = 5
num_samples = len(ds['train'])
examples = "[INST] The following are examples of pairs of hyp and tgt sentences that can either be Hallucination or Not Hallucination depending on how similar they are [\INST]\n"

for i in range(num_examples):
    num = int(random() * num_samples)
    hyp = ds['train'][num]['hyp']
    tgt = ds['train'][num]['tgt']
    label = ds['train'][num]['label']
    examples += f"\nExample {i+1}\n<HYP>: {hyp}\n<TGT>: {tgt}\n<LABEL>: {label}\n"
print(examples)

[INST] The following are examples of pairs of hyp and tgt sentences that can either be Hallucination or Not Hallucination depending on how similar they are [\INST]

Example 1
<HYP>: There's no way this is going to happen.
<TGT>: This can't happen.
<LABEL>: Hallucination

Example 2
<HYP>: Take a close look at it.
<TGT>: Take a good look.
<LABEL>: Not Hallucination

Example 3
<HYP>: What do you think I'm, like, a monster, like, "A monster"?
<TGT>: You think I'm some kind of a monster?
<LABEL>: Hallucination

Example 4
<HYP>: We don't have the money to risk it, all right?
<TGT>: We can't risk that.
<LABEL>: Hallucination

Example 5
<HYP>: You're going somewhere else?
<TGT>: What's your point?
<LABEL>: Hallucination



In [5]:
def generate_prompt(mapped_ds):
    hyp = mapped_ds["hyp"]
    tgt = mapped_ds["tgt"] if mapped_ds['ref'] != "src" else mapped_ds['src']
    prompt = f"{examples}\n Example {num_examples+1}\n<HYP>: {hyp}\n<TGT>: {tgt}\n<LABEL>: "
    return {'prompts' : prompt}

In [6]:
ds = ds.map(generate_prompt)
ds

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'label', 'model', 'ref', 'hyp', 'task', 'tgt', 'p(Hallucination)', 'src', 'prompts'],
        num_rows: 125
    })
})

In [7]:
def generate_prediction(mapped_ds):
    input_ids = tokenizer(mapped_ds['prompts'], padding=True, truncation=True, max_length=500, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.01, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=5)
    decoded_output = tokenizer.batch_decode(output)
    return {'output' : decoded_output}

In [8]:
ds = ds.map(generate_prediction, batched=True, batch_size=BATCH_SIZE)
ds

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

/data1/malto/fborra/venv/lib/python3.9/site-packages/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['labels', 'label', 'model', 'ref', 'hyp', 'task', 'tgt', 'p(Hallucination)', 'src', 'prompts', 'output'],
        num_rows: 125
    })
})

In [9]:
def extract_synthetic_label(mapped_ds):
    syn_labels = []
    for item in mapped_ds['output']:
        if "Not" in item.splitlines()[-1]:
            syn_labels.append("Not Hallucination")
        else:
            syn_labels.append("Hallucination")
    return {'synthetic_labels' : syn_labels}

In [10]:
ds = ds.map(extract_synthetic_label, batched=True)
ds

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'label', 'model', 'ref', 'hyp', 'task', 'tgt', 'p(Hallucination)', 'src', 'prompts', 'output', 'synthetic_labels'],
        num_rows: 125
    })
})

In [11]:
correct = 0
for a, b in zip(ds['train']['label'], ds['train']['synthetic_labels']):
    if a == b:
        correct += 1
correct / num_samples

0.584

In [12]:
ds['train'][7]

{'labels': ['Not Hallucination',
  'Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination'],
 'label': 'Not Hallucination',
 'model': '',
 'ref': 'either',
 'hyp': "That's what he's saying.",
 'task': 'PG',
 'tgt': 'Yeah, he said that.',
 'p(Hallucination)': 0.2,
 'src': "That's what he said.",
 'prompts': '[INST] The following are examples of pairs of hyp and tgt sentences that can either be Hallucination or Not Hallucination depending on how similar they are [\\INST]\n\nExample 1\n<HYP>: There\'s no way this is going to happen.\n<TGT>: This can\'t happen.\n<LABEL>: Hallucination\n\nExample 2\n<HYP>: Take a close look at it.\n<TGT>: Take a good look.\n<LABEL>: Not Hallucination\n\nExample 3\n<HYP>: What do you think I\'m, like, a monster, like, "A monster"?\n<TGT>: You think I\'m some kind of a monster?\n<LABEL>: Hallucination\n\nExample 4\n<HYP>: We don\'t have the money to risk it, all right?\n<TGT>: We can\'t risk that.\n<LABEL>: Hallucination\n\nExampl

## Create Labels for New Data Points

In [13]:
def create_empty_column():
    tr_ds = load_dataset("json", data_files=[str(BASE_DIR / "train.model-agnostic.json")])
    tr_ds = tr_ds.map(lambda x: {'labels' : []})
    tr_ds['train'] = tr_ds['train'].filter(lambda x: x['task'] == TASK_TYPE)
    tr_ds['train'].to_json(str(BASE_DIR / f"train_labeled_{TASK_TYPE}_SOLAR.model-agnostic.json"))

#create_empty_column()

In [14]:
tr_ds = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_{TASK_TYPE}_SOLAR.model-agnostic.json")])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
tr_ds = tr_ds.map(generate_prompt)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [16]:
tr_ds = tr_ds.map(generate_prediction, batched=True, batch_size=BATCH_SIZE)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [17]:
tr_ds = tr_ds.map(extract_synthetic_label, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
def add_prediction(mapped_ds):
    mapped_ds['labels'].append(mapped_ds['synthetic_labels'])
    return mapped_ds

tr_ds = tr_ds.map(add_prediction)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
tr_ds['train'][11]

{'hyp': '♪ I got the joy, joy, joy, joy Down in my heart ♪',
 'src': 'I got the joy, joy, joy, joy Down in my heart',
 'task': 'PG',
 'ref': 'src',
 'tgt': '',
 'model': '',
 'labels': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination'],
 'prompts': '[INST] The following are examples of pairs of hyp and tgt sentences that can either be Hallucination or Not Hallucination depending on how similar they are [\\INST]\n\nExample 1\n<HYP>: There\'s no way this is going to happen.\n<TGT>: This can\'t happen.\n<LABEL>: Hallucination\n\nExample 2\n<HYP>: Take a close look at it.\n<TGT>: Take a good look.\n<LABEL>: Not Hallucination\n\nExample 3\n<HYP>: What do you think I\'m, like, a monster, like, "A monster"?\n<TGT>: You think I\'m some kind of a monster?\n<LABEL>: Hallucination\n\nExample 4\n<HYP>: We don\'t have the money to risk it, all right?\n<TGT>: We can\'t risk that.\n<LABEL>: Hallucination\n\nExample 5\n<HYP>: You\'re going

In [20]:
tr_ds['train'].to_json(str(BASE_DIR / f"train_labeled_{TASK_TYPE}_SOLAR.model-agnostic.json"))

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

20068245